# Survival Analysis in Glioma from Selected Features

This is a very rough exploration of survival biomarkers obtained from the selected features from the Glioma classifier evaluation. Using Cox regression on individual biomarkers and Kaplan-Meier analysis, the aim is to obtain biomarkers that are linked to disease progression.

In [1]:
!conda env create -f environment.yml
!pip install ipykernel
!pip install lifelines
!python -m ipykernel install --user --name JupyterLab --display-name "Python (JupyterLab)"


CondaValueError: prefix already exists: /home/talal/anaconda3/envs/JupyterLab

  Preparing metadata (setup.py) ... done
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4032 sha256=57ec029abd32e5370e2feaa79cdd9341d74928c6ec83029248331a7f828fc34a
  Stored in directory: /home/talal/.cache/pip/wheels/a8/03/64/8557323821d25118c3a2dc1646996f7a962a8970d4b7d22473
Successfully built autograd-gamma
Installed kernelspec JupyterLab in /home/talal/.local/share/jupyter/kernels/jupyterlab


In [3]:
import os
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Read the datasets

In [ ]:
# Verify the current directory
print("Current Directory:", os.getcwd())

In [5]:
# load the dataset
extraction_dir = "./processed"
data_path = os.path.join(extraction_dir, 'selected_feature_data.csv')
labels_path = os.path.join(extraction_dir, 'glioma_labels.csv')
data_df = pd.read_csv(data_path)
labels_df = pd.read_csv(labels_path)

print("Gene Expression Data Shape:\n", data_df.shape)
print("\nLabels Head:\n", labels_df.shape)

Gene Expression Data Shape:
 (184, 201)

Labels Head:
 (184, 2)


In [4]:
# Load clinical data 
clinical_data = pd.read_csv(os.path.join(os.getcwd(),"metadata/clinical.tsv"), sep='\t')
clinical_data.head()

,case_id,case_submitter_id,project_id,age_at_index,age_is_obfuscated,cause_of_death,cause_of_death_source,country_of_residence_at_enrollment,days_to_birth,days_to_death,...,treatment_arm,treatment_dose,treatment_dose_units,treatment_effect,treatment_effect_indicator,treatment_frequency,treatment_intent_type,treatment_or_therapy,treatment_outcome,treatment_type
0,d420e653-3fb2-432b-9e81-81232a80264d,HCM-BROD-0210-C71,HCMI-CMDC,'--,False,Cancer Related,'--,'--,-19586,481,...,'--,'--,'--,'--,'--,'--,Adjuvant,no,'--,Immunotherapy (Including Vaccines)
1,d420e653-3fb2-432b-9e81-81232a80264d,HCM-BROD-0210-C71,HCMI-CMDC,'--,False,Cancer Related,'--,'--,-19586,481,...,'--,'--,'--,'--,'--,'--,Adjuvant,no,'--,Targeted Molecular Therapy
2,d420e653-3fb2-432b-9e81-81232a80264d,HCM-BROD-0210-C71,HCMI-CMDC,'--,False,Cancer Related,'--,'--,-19586,481,...,'--,'--,'--,'--,'--,'--,Adjuvant,no,'--,"Radiation Therapy, NOS"
3,d420e653-3fb2-432b-9e81-81232a80264d,HCM-BROD-0210-C71,HCMI-CMDC,'--,False,Cancer Related,'--,'--,-19586,481,...,'--,'--,'--,'--,'--,'--,Adjuvant,yes,'--,Chemotherapy
4,d420e653-3fb2-432b-9e81-81232a80264d,HCM-BROD-0210-C71,HCMI-CMDC,'--,False,Cancer Related,'--,'--,-19586,481,...,'--,'--,'--,'--,'--,'--,Neoadjuvant,no,'--,'--


In [10]:
clinical_data.shape

(1386, 158)

In [11]:
# drop duplicates based on 'case_submitter_id'
clinical_data_dedup = clinical_data.drop_duplicates(subset='case_submitter_id', keep='first')
clinical_data_dedup.shape

(663, 158)

In [12]:
labels_df = labels_df.rename(columns={"Sample ID": "sample", "label":"cancer_type"})
labels_df.head()

,sample,cancer_type
0,HCM-BROD-0210-C71-85R,Glioblastoma
1,HCM-BROD-0209-C71-85A,Glioblastoma
2,C3L-03266-01,Glioblastoma
3,C3L-03727-01,Glioblastoma
4,C3L-01887-01,Glioblastoma


### Create survival dataframe

In [13]:
# check to identify potential time-to-event or survival-related columns
clinical_data_dedup.columns.tolist()

['case_id',
 'case_submitter_id',
 'project_id',
 'age_at_index',
 'age_is_obfuscated',
 'cause_of_death',
 'cause_of_death_source',
 'country_of_residence_at_enrollment',
 'days_to_birth',
 'days_to_death',
 'ethnicity',
 'gender',
 'occupation_duration_years',
 'premature_at_birth',
 'race',
 'vital_status',
 'weeks_gestation_at_birth',
 'year_of_birth',
 'year_of_death',
 'adrenal_hormone',
 'age_at_diagnosis',
 'ajcc_clinical_m',
 'ajcc_clinical_n',
 'ajcc_clinical_stage',
 'ajcc_clinical_t',
 'ajcc_pathologic_m',
 'ajcc_pathologic_n',
 'ajcc_pathologic_stage',
 'ajcc_pathologic_t',
 'ajcc_staging_system_edition',
 'anaplasia_present',
 'anaplasia_present_type',
 'ann_arbor_b_symptoms',
 'ann_arbor_b_symptoms_described',
 'ann_arbor_clinical_stage',
 'ann_arbor_extranodal_involvement',
 'ann_arbor_pathologic_stage',
 'best_overall_response',
 'breslow_thickness',
 'burkitt_lymphoma_clinical_variant',
 'child_pugh_classification',
 'circumferential_resection_margin',
 'classificatio

Here, 'case_submitter_id', 'days_to_death', 'days_to_last_follow_up', 'vital_status' seem appropriate.

In [14]:
survival_df = clinical_data_dedup[['case_submitter_id', 'days_to_death', 'days_to_last_follow_up', 'vital_status']].copy()

In [15]:
vital_status_distribution = survival_df['vital_status'].value_counts()
vital_status_distribution

vital_status
Alive           385
Dead            274
Not Reported      3
Unknown           1
Name: count, dtype: int64

Dropping the 'Not Reported' and 'Unknown' labels

In [16]:
# remove rows where 'vital_status' is 'Not Reported' or 'Unknown'
survival_df = survival_df[~survival_df['vital_status'].isin(['Not Reported', 'Unknown'])]
# create the 'event' column (1 = Dead, 0 = Alive)
survival_df['event'] = survival_df['vital_status'].apply(lambda x: 1 if x == 'Dead' else 0)

# 'days_to_death' is missing, use 'days_to_last_follow_up' as the time
survival_df['days_to_death'] = pd.to_numeric(survival_df['days_to_death'], errors='coerce')
survival_df['days_to_last_follow_up'] = pd.to_numeric(survival_df['days_to_last_follow_up'], errors='coerce')
survival_df['time'] = survival_df['days_to_death'].fillna(survival_df['days_to_last_follow_up'])
survival_df = survival_df.dropna(subset=['time'])
survival_df['time'].isnull().sum()

np.int64(0)

In [17]:
data_df['sample_id_short'] = data_df['sample'].str.split('-').str[:3].str.join('-')

merged_survival_df = pd.merge(
    data_df, 
    survival_df, 
    left_on='sample_id_short', 
    right_on='case_submitter_id',
    how='inner'
)


merged_survival_df = pd.merge(
    merged_survival_df, 
    labels_df, 
    left_on='sample', 
    right_on='sample',
    how='inner'
)

merged_survival_df = merged_survival_df.set_index('sample')
merged_survival_df

,IRF3,TRIM67,FTH1P1,USP27X,AL162430.1,MAGOH,AC079140.2,NFIA-AS2,ABHD17AP3,KRT8P30,...,AC034207.1,AC004057.1,sample_id_short,case_submitter_id,days_to_death,days_to_last_follow_up,vital_status,event,time,cancer_type
sample,,,,,,,,,,,,,,,,,,,,,
TCGA-32-2634-01A,0.359426,-0.418560,0.278336,0.004363,0.470171,1.792828,0.149927,-0.636651,2.396006,-0.548862,...,0.025250,0.338042,TCGA-32-2634,TCGA-32-2634,NaN,693.0,Alive,0,693.0,Glioblastoma
TCGA-12-3652-01A,2.906740,-0.464853,-0.249051,-1.145257,1.424057,0.875777,1.020365,-0.229359,2.640269,-0.431305,...,0.823319,0.681251,TCGA-12-3652,TCGA-12-3652,1062.0,1050.0,Dead,1,1062.0,Glioblastoma
TCGA-06-0168-01A,0.052822,-0.451306,3.268133,-0.470688,2.852868,1.334454,3.802686,0.170225,2.251226,-0.483002,...,2.395304,0.743306,TCGA-06-0168,TCGA-06-0168,598.0,579.0,Dead,1,598.0,Glioblastoma
TCGA-12-1597-01B,0.400325,-0.352738,0.988660,-0.864894,0.841592,0.046578,0.628687,-0.614537,0.733531,-0.151468,...,0.696503,0.293637,TCGA-12-1597,TCGA-12-1597,675.0,427.0,Dead,1,675.0,Glioblastoma
TCGA-28-2513-01A,0.208342,-0.464919,0.533711,-1.008034,0.360817,0.317250,0.592372,0.392751,1.138038,-0.714356,...,0.104382,0.664128,TCGA-28-2513,TCGA-28-2513,NaN,222.0,Alive,0,222.0,Glioblastoma
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-DU-A7TC-01A,-0.229351,-0.107087,-0.577640,0.108821,-0.650292,-0.397045,-0.780429,-0.589524,-0.488029,-0.119545,...,-0.489778,-0.244460,TCGA-DU-A7TC,TCGA-DU-A7TC,NaN,1137.0,Alive,0,1137.0,Astrocytoma
TCGA-HT-7857-01A,0.103622,-0.459083,-0.475231,-0.918176,-0.723850,-0.355781,-0.694412,-0.579958,-0.515304,-0.714356,...,-0.524757,-0.293841,TCGA-HT-7857,TCGA-HT-7857,NaN,7.0,Alive,0,7.0,Astrocytoma
TCGA-TQ-A7RV-02A,-0.365392,-0.444735,-0.681603,-0.962431,-0.787929,-0.843485,-0.517286,0.133663,-0.633783,-0.155662,...,-0.573617,-0.284562,TCGA-TQ-A7RV,TCGA-TQ-A7RV,NaN,1868.0,Alive,0,1868.0,Astrocytoma


In [18]:
# Extract the survival info
survival_info = merged_survival_df[['time', 'event']]
biomarkers_df = merged_survival_df.copy()
columns_to_drop = ['sample_id_short', 'case_submitter_id', 'days_to_death', 'days_to_last_follow_up', 'vital_status', 'event', 'time', 'cancer_type']
biomarkers_df.drop(columns = columns_to_drop, axis=1)
biomarkers_df 

,IRF3,TRIM67,FTH1P1,USP27X,AL162430.1,MAGOH,AC079140.2,NFIA-AS2,ABHD17AP3,KRT8P30,...,AC034207.1,AC004057.1,sample_id_short,case_submitter_id,days_to_death,days_to_last_follow_up,vital_status,event,time,cancer_type
sample,,,,,,,,,,,,,,,,,,,,,
TCGA-32-2634-01A,0.359426,-0.418560,0.278336,0.004363,0.470171,1.792828,0.149927,-0.636651,2.396006,-0.548862,...,0.025250,0.338042,TCGA-32-2634,TCGA-32-2634,NaN,693.0,Alive,0,693.0,Glioblastoma
TCGA-12-3652-01A,2.906740,-0.464853,-0.249051,-1.145257,1.424057,0.875777,1.020365,-0.229359,2.640269,-0.431305,...,0.823319,0.681251,TCGA-12-3652,TCGA-12-3652,1062.0,1050.0,Dead,1,1062.0,Glioblastoma
TCGA-06-0168-01A,0.052822,-0.451306,3.268133,-0.470688,2.852868,1.334454,3.802686,0.170225,2.251226,-0.483002,...,2.395304,0.743306,TCGA-06-0168,TCGA-06-0168,598.0,579.0,Dead,1,598.0,Glioblastoma
TCGA-12-1597-01B,0.400325,-0.352738,0.988660,-0.864894,0.841592,0.046578,0.628687,-0.614537,0.733531,-0.151468,...,0.696503,0.293637,TCGA-12-1597,TCGA-12-1597,675.0,427.0,Dead,1,675.0,Glioblastoma
TCGA-28-2513-01A,0.208342,-0.464919,0.533711,-1.008034,0.360817,0.317250,0.592372,0.392751,1.138038,-0.714356,...,0.104382,0.664128,TCGA-28-2513,TCGA-28-2513,NaN,222.0,Alive,0,222.0,Glioblastoma
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-DU-A7TC-01A,-0.229351,-0.107087,-0.577640,0.108821,-0.650292,-0.397045,-0.780429,-0.589524,-0.488029,-0.119545,...,-0.489778,-0.244460,TCGA-DU-A7TC,TCGA-DU-A7TC,NaN,1137.0,Alive,0,1137.0,Astrocytoma
TCGA-HT-7857-01A,0.103622,-0.459083,-0.475231,-0.918176,-0.723850,-0.355781,-0.694412,-0.579958,-0.515304,-0.714356,...,-0.524757,-0.293841,TCGA-HT-7857,TCGA-HT-7857,NaN,7.0,Alive,0,7.0,Astrocytoma
TCGA-TQ-A7RV-02A,-0.365392,-0.444735,-0.681603,-0.962431,-0.787929,-0.843485,-0.517286,0.133663,-0.633783,-0.155662,...,-0.573617,-0.284562,TCGA-TQ-A7RV,TCGA-TQ-A7RV,NaN,1868.0,Alive,0,1868.0,Astrocytoma


# Cox Analysis
#### Global Analysis

In [19]:
from lifelines import CoxPHFitter

significant_biomarkers = []
survival_info = merged_survival_df[['time', 'event', 'cancer_type']]

# perform univariate Cox analysis for each biomarker
for biomarker in biomarkers_df.columns:
    # Create a temporary dataframe with just the biomarker and survival information
    temp_data = pd.concat([biomarkers_df[biomarker], survival_info[['time', 'event']]], axis=1)
    
    # Initialize Cox model
    cox = CoxPHFitter()
    
    # Fit the univariate Cox model with subtype stratification
    try:
        cox.fit(temp_data, duration_col='time', event_col='event')
        # If the p-value is below 0.01, keep the biomarker
        if cox.summary['p'].values[0] < 0.01:
            significant_biomarkers.append(biomarker)
    except:
        continue

# Create a reduced dataset with significant biomarkers
reduced_biomarker_data = biomarkers_df[significant_biomarkers]

print(f"Number of significant biomarkers: {len(significant_biomarkers)}")

/home/talal/anaconda3/envs/JupyterLab/lib/python3.9/site-packages/lifelines/fitters/coxph_fitter.py:1614: ConvergenceWarning: Newton-Raphson failed to converge sufficiently. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-model
  warnings.warn(


Number of significant biomarkers: 164


Note: This is a hold-over when i considered no subtype, but I still kept this analysis:

High number of significant biomarkers (= 195) suggests something's odd. As per the [this site](https://online.stat.psu.edu/stat462/node/180/), it could be multi-collinearity. It might be worth using regularization methods like Ridge (L2) or ElasticNet regularization, which can handle multicollinearity by shrinking the coefficients of less important biomarkers. 

### Strata - Subtype

In [20]:
significant_biomarkers = []
survival_info = merged_survival_df[['time', 'event', 'cancer_type']]

# perform univariate Cox analysis for each biomarker
for biomarker in biomarkers_df.columns:
    # Create a temporary dataframe with just the biomarker and survival information
    temp_data = pd.concat([biomarkers_df[biomarker], survival_info[['time', 'event', 'cancer_type']]], axis=1)
    
    # Initialize Cox model
    cox = CoxPHFitter()
    
    # Fit the univariate Cox model with subtype stratification
    try:
        cox.fit(temp_data, duration_col='time', event_col='event', strata='cancer_type')
        # If the p-value is below 0.01, keep the biomarker
        if cox.summary['p'].values[0] < 0.01:
            significant_biomarkers.append(biomarker)
    except:
        continue

# Create a reduced dataset with significant biomarkers
reduced_biomarker_data = biomarkers_df[significant_biomarkers]

print(f"Number of significant biomarkers: {len(significant_biomarkers)}")

Number of significant biomarkers: 3


In [21]:
correlation_matrix = reduced_biomarker_data.corr().abs()
upper = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.8)]
print(f"Columns to drop due to multicollinearity: {to_drop}")

# Drop the highly correlated biomarkers
reduced_biomarker_data = reduced_biomarker_data.drop(columns=to_drop)

Columns to drop due to multicollinearity: []


In [22]:
cox_model_significant = CoxPHFitter(penalizer=0.1, l1_ratio=0.2) # Fit the ElasticNet Cox model with both L1 and L2 regularization
cox_model_significant.fit(pd.concat([reduced_biomarker_data, survival_info], axis=1), duration_col='time', event_col='event', strata='cancer_type')

# Print the summary of the model
cox_model_significant.print_summary()

model,lifelines.CoxPHFitter
duration col,'time'
event col,'event'
penalizer,0.1
l1 ratio,0.2
strata,cancer_type
baseline estimation,breslow
number of observations,163
number of events observed,73
partial log-likelihood,-203.65
time fit was run,2025-02-20 23:32:12 UTC


In [23]:
#  biomarkers based on p-value < 0.05
significant_biomarkers = cox_model_significant.summary[cox_model_significant.summary['p'] < 0.05].index.tolist()
reduced_biomarker_data_significant = reduced_biomarker_data[significant_biomarkers]
cox_model_significant =  CoxPHFitter(penalizer = 0.1, l1_ratio = 0.2) 
cox_model_significant.fit(pd.concat([reduced_biomarker_data_significant, survival_info], axis=1), duration_col='time', event_col='event', strata='cancer_type')
cox_model_significant.print_summary()

model,lifelines.CoxPHFitter
duration col,'time'
event col,'event'
penalizer,0.1
l1 ratio,0.2
strata,cancer_type
baseline estimation,breslow
number of observations,163
number of events observed,73
partial log-likelihood,-207.96
time fit was run,2025-02-20 23:32:12 UTC


TGIF1 demonstrates a significant association with survival(p-value < 0.005). The hazard ratio of 1.40 suggests that for every unit increase in TGIF1 expression, there is a 40% increase in the hazard rate, implying worse survival outcomes. However, the model's concordance score of 0.50 reflects poor predictive; model's power to correctly discriminate between patients' survival times is limited.

This biomarker is already documented with higher with worse survival outcomes as this paper.
- Wang, B., Ma, Q., Wang, X., Guo, K., Liu, Z., & Li, G. (2022). TGIF1 overexpression promotes glioma progression and worsens patient prognosis. Cancer medicine, 11(24), 5113–5128. https://doi.org/10.1002/cam4.4822)